In [5]:
import math

# ====== Down-and-Out Call Option Delta via Finite Difference ======
# Formula:
# V(S,K,t) = C(S,K,t) - (B/S)^(2a) * C(B^2/S, K, t)
# a = (r - q)/sigma^2 - 1/2
# Delta ≈ (V(S+ΔS) - V(S-ΔS)) / (2ΔS)

# ====== Given Parameters ======
S = 40.0        # Spot price
K = 40.0        # Strike price
t = 0.5         # Time to maturity (years)
sigma = 0.30    # Volatility (30%)
r = 0.05        # Risk-free rate (5%)
q = 0.0         # Dividend yield (non-dividend-paying asset)
B = 35.0        # Barrier
dS = 0.0001       # Small price increment for finite difference

# ====== Helper functions ======
def Phi(x):
    """Standard normal CDF"""
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))

def bs_call_price(S_, K_, t_, r_, sigma_, q_=0.0):
    """Black-Scholes European call price"""
    if t_ <= 0 or S_ <= 0 or K_ <= 0:
        return max(S_ - K_, 0.0)
    sqrt_t = math.sqrt(t_)
    d1 = (math.log(S_ / K_) + (r_ - q_ + 0.5 * sigma_**2) * t_) / (sigma_ * sqrt_t)
    d2 = d1 - sigma_ * sqrt_t
    return S_ * math.exp(-q_ * t_) * Phi(d1) - K_ * math.exp(-r_ * t_) * Phi(d2)

def down_and_out_call(S_, K_, t_, r_, sigma_, q_, B_):
    """Analytical price of a down-and-out call using reflection principle"""
    a = (r_ - q_) / (sigma_**2) - 0.5
    C1 = bs_call_price(S_, K_, t_, r_, sigma_, q_)
    S_prime = (B_**2) / S_
    C2 = bs_call_price(S_prime, K_, t_, r_, sigma_, q_)
    multiplier = (B_ / S_) ** (2.0 * a)
    return C1 - multiplier * C2

# ====== Compute central finite difference Delta ======
V_plus  = down_and_out_call(S + dS, K, t, r, sigma, q, B)
V = down_and_out_call(S, K, t, r, sigma, q, B)
V_minus = down_and_out_call(S - dS, K, t, r, sigma, q, B)
Delta_fd = (V_plus - V_minus) / (2 * dS)

# ====== Also compute base price ======
V0 = down_and_out_call(S, K, t, r, sigma, q, B)

# ====== Output ======
print("===== Down-and-Out Call Option Delta (Finite Difference) =====")
print(f"S = {S}, K = {K}, B = {B}, t = {t}, sigma = {sigma}, r = {r}")
print(f"Base option value V(S) = {V0:.6f}")
print(f"V(S+ΔS) = {V_plus:.6f}, V(S-ΔS) = {V_minus:.6f}")
print(f"ΔS = {dS}")
print(f"Finite difference Delta ≈ {Delta_fd:.6f}")


===== Down-and-Out Call Option Delta (Finite Difference) =====
S = 40.0, K = 40.0, B = 35.0, t = 0.5, sigma = 0.3, r = 0.05
Base option value V(S) = 3.398883
V(S+ΔS) = 3.398954, V(S-ΔS) = 3.398813
ΔS = 0.0001
Finite difference Delta ≈ 0.703253
